In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
from datetime import date, time, datetime
import datetime

In [2]:
# сохраняем данные
data = pd.read_excel('/Users/yana_shinkaryukmail.ru/Downloads/bf_raul.xls')
data.head()

,Unnamed: 0,id_кандидата,пол,возраст,дата_добавления,инвалидность,сиротский_опыт,адаптированная_образовательная_программа,начало_трудоустройства,завершение_трудоустройства,1+_месяц_работы,тип_площадки,тип_трудоустройства,тип_опыта_работы,регион_код,провайдер_код
0,0,1,Женский,25,2018-12-06,ИСТИНА,ЛОЖЬ,ЛОЖЬ,2023-01-16,NaT,ИСТИНА,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500
1,1,2,Мужской,28,2021-09-22,ИСТИНА,ЛОЖЬ,ЛОЖЬ,2023-01-09,NaT,ИСТИНА,Внешняя,Открытый рынок с сопровождением,Повторный,reg200,prov3500
2,2,3,Женский,31,2016-07-08,ИСТИНА,ЛОЖЬ,ЛОЖЬ,2023-04-18,2023-06-30,ИСТИНА,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500
3,3,4,Мужской,34,2022-10-07,ИСТИНА,ЛОЖЬ,ЛОЖЬ,2022-11-21,2022-12-30,ИСТИНА,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500
4,4,4,Мужской,34,2022-10-07,ИСТИНА,ЛОЖЬ,ЛОЖЬ,2023-03-22,2023-03-30,ЛОЖЬ,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500


## Предобработка данных

In [3]:
# удаляем столбец с номером строки
data = data.drop(['Unnamed: 0'], axis=1)

In [4]:
# проверим типы данных
data.dtypes

id_кандидата                                         int64
пол                                                 object
возраст                                              int64
дата_добавления                             datetime64[ns]
инвалидность                                        object
сиротский_опыт                                      object
адаптированная_образовательная_программа            object
начало_трудоустройства                      datetime64[ns]
завершение_трудоустройства                  datetime64[ns]
1+_месяц_работы                                     object
тип_площадки                                        object
тип_трудоустройства                                 object
тип_опыта_работы                                    object
регион_код                                          object
провайдер_код                                       object
dtype: object

In [5]:
# проверим данные на наличие дубликатов
data.duplicated().sum()

0

## Создание вычисляемых полей

In [6]:
# автоматизируем расчет переменной в столбце "1+_месяц_работы", 
# для сотрудников, работающих более 1 месяца 
data['1+_месяц_работы'] = data['завершение_трудоустройства'].isna() & \
            (pd.to_datetime(date.today()) - data['начало_трудоустройства'] > datetime.timedelta(weeks=4)) | \
            (data['завершение_трудоустройства'] > data['начало_трудоустройства'] + datetime.timedelta(weeks=4))

In [7]:
# посчитаем продолжительность трудоустройства кандидатов, у которых есть дата завершения трудоустройства
data['отработано_дней'] = data['завершение_трудоустройства'] - data['начало_трудоустройства']

In [8]:
# посчитаем продолжительность трудоустройства кандидатов без даты завершения трудоустройства
data['отработано_дней'].fillna(
    value=pd.to_datetime(date.today()) - data['начало_трудоустройства'], inplace=True)

In [9]:
# добавим информацию о количестве полных месяцев, отработанных кандидатами 
data['отработано_полных_месяцев'] = round(data['отработано_дней']  / np.timedelta64(1, 'M'))

## Замена булевых переменных   
Заменим значения True/False и ИСТИНА/ЛОЖЬ на "Да"/"Нет":

In [10]:
data['1+_месяц_работы'] = data['1+_месяц_работы'].apply(lambda x: "Да" if x else "Нет")

In [11]:
data['инвалидность'].replace({'ИСТИНА':'Да', 
                              'ЛОЖЬ': 'Нет'}, inplace=True)

In [12]:
data['сиротский_опыт'].replace({'ИСТИНА':'Да', 
                                'ЛОЖЬ': 'Нет'}, inplace=True)

In [13]:
data['адаптированная_образовательная_программа'].replace({'ИСТИНА':'Да', 
                                                          'ЛОЖЬ': 'Нет'}, inplace=True)

In [15]:
data.head()

,id_кандидата,пол,возраст,дата_добавления,инвалидность,сиротский_опыт,адаптированная_образовательная_программа,начало_трудоустройства,завершение_трудоустройства,1+_месяц_работы,тип_площадки,тип_трудоустройства,тип_опыта_работы,регион_код,провайдер_код,отработано_дней,отработано_полных_месяцев
0,1,Женский,25,2018-12-06,Да,Нет,Нет,2023-01-16,NaT,Да,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500,326 days,11.0
1,2,Мужской,28,2021-09-22,Да,Нет,Нет,2023-01-09,NaT,Да,Внешняя,Открытый рынок с сопровождением,Повторный,reg200,prov3500,333 days,11.0
2,3,Женский,31,2016-07-08,Да,Нет,Нет,2023-04-18,2023-06-30,Да,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500,73 days,2.0
3,4,Мужской,34,2022-10-07,Да,Нет,Нет,2022-11-21,2022-12-30,Да,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500,39 days,1.0
4,4,Мужской,34,2022-10-07,Да,Нет,Нет,2023-03-22,2023-03-30,Нет,Внешняя,Открытый рынок без сопровождения,Повторный,reg200,prov3500,8 days,0.0


In [17]:
# cохраним итоговый файл в формате csv
data.to_csv(r'/Users/yana_shinkaryukmail.ru/Downloads/bf_raul.csv', index= False)